In [1]:
import os, sys
sys.path.append("..")

from src.milvus_store import MilvusStore
from src.config import ConfigLoader
from src.agent import AgenticRAG

d:\Utilities\miniconda\envs\agen\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-18 21:07:48,043 - src.milvus_store - INFO - Logging to file: logs\milvus_store.log
2025-10-18 21:07:50,399 - src.agent - INFO - Logging to file: logs/agent.log


In [2]:
config = ConfigLoader("../config.yaml")

In [3]:
config.set("database", "collection_name", "doc_2_db")

uri = config.get("database", "uri", default="http://localhost:19530")
db_name = config.get("database", "name", default="gil")
collection_name = config.get("database", "collection_name", default="multimodal_rag")
embed_model = config.get("model", "embeddings", default="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
# namespace = config.get("database", "namespace", default="viettel")


vs1 = {"store": MilvusStore(
            uri=uri,
            db_name=db_name,
            collection_name=collection_name,
            drop_old=False,
            #   namespace=namespace
            ),
        "name": "retrieve_documents_on_docling",
        "description": "Search and retrieve information from the document collection on a topic of docling",
        "k": config.get("retrieval", "k", default=3),
        "ranker_weights": config.get("retrieval", "weights", default=[0.6, 0.4])
    }

vector_stores = [vs1]

2025-10-18 21:07:50,652 - src.milvus_store - INFO - Database 'gil' already exists.


In [4]:
# milvus_store.similarity_search_with_score(question)

In [5]:
agent = AgenticRAG(vector_stores=vector_stores)

2025-10-18 21:08:15,426 - src.milvus_store - INFO - [MilvusStore] Built filter expr: namespace == "viettel"
2025-10-18 21:08:15,427 - src.milvus_store - INFO - [MilvusStore] as_retriever expr=namespace == "viettel", k=3, fetch_k=12
2025-10-18 21:08:17,472 - src.agent - INFO - AgenticRAG initialized with model: qwen3:1.7b and 1 vector store(s)


In [7]:
results = agent.run_mcq_csv("../test.csv")

2025-10-18 21:10:26,988 - src.agent - INFO - Thread ID updated to: 1760796626988_a88850dbf9914c3cad5fcf5151a0da50
2025-10-18 21:10:26,989 - src.agent - INFO - [trace] run_mcq called with question: Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?...
2025-10-18 21:10:26,990 - src.agent - INFO - [trace] run_mcq options: ['A', 'B', 'C', 'D']
2025-10-18 21:10:26,990 - src.agent - INFO - [trace] run_mcq content: Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?
A: Lưu trữ dữ liệu trên máy chủ
B: Kết nối Internet và quản lý thiết bị từ xa
C: Cung cấp dịch vụ phân tích dữ liệu lớn
D: Thay thế hoàn toàn...
2025-10-18 21:10:26,991 - src.agent - INFO - Thread ID updated to: 1760796626991_70c51deb94144f558feb0aa176b13b34
2025-10-18 21:10:26,991 - src.agent - INFO - [trace] run_mcq invoking graph...
2025-10-18 21:10:26,998 - src.agent - INFO - [trace] _generate_query_or_respond called
2025-10-18 21:10:26,999 - src.agent - INFO - [trace] state.messages count: 1
2025-10-18 21:10:26

In [8]:
from pprint import pprint
import re

In [9]:
results

[{'question': 'Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?',
  'A': 'Lưu trữ dữ liệu trên máy chủ',
  'B': 'Kết nối Internet và quản lý thiết bị từ xa',
  'C': 'Cung cấp dịch vụ phân tích dữ liệu lớn',
  'D': 'Thay thế hoàn toàn điện toán đám mây',
  'response': 'Error: Connection error.'},
 {'question': 'Năm 2010, công nghệ nào thường được sử dụng trong kiểm soát ra vào ngôi nhà thông minh?',
  'A': 'Bluetooth Low Energy',
  'B': 'RFID và nhận dạng khuôn mặt/vân tay',
  'C': 'Zigbee',
  'D': 'Blockchain',
  'response': 'Error: Connection error.'},
 {'question': 'Bộ cảm biến trong nhà thông minh thực hiện chức năng gì và bộ truyền động có vai trò gì?',
  'A': 'Bộ cảm biến hiển thị dữ liệu, bộ truyền động thu thập dữ liệu',
  'B': 'Bộ cảm biến lưu trữ dữ liệu, bộ truyền động hiển thị dữ liệu',
  'C': 'Bộ cảm biến thu thập dữ liệu, bộ truyền động thực hiện lệnh từ máy chủ hoặc thiết bị điều khiển',
  'D': 'Bộ cảm biến điều khiển thiết bị, bộ truyền động lưu trữ dữ liệu',
 

In [10]:
for r in results:
    response = r["response"]

    # --- Tách phần <think> ... </think>
    thinking = re.search(r"<think>(.*?)</think>", response, re.DOTALL)
    thinking_text = thinking.group(1).strip() if thinking else ""

    # --- Phần còn lại sau </think>
    remainder = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()

    # --- Tách rationale và final answer (nếu có)
    rationale_match = re.search(r"Rationale:(.*?)(?:Final Answer:|$)", remainder, re.DOTALL)
    rationale_text = rationale_match.group(1).strip() if rationale_match else ""

    final_match = re.search(r"Final Answer:\s*(.*)", remainder)
    final_answer = final_match.group(1).strip() if final_match else ""

    # --- In ra kết quả
    print(f"\n❓ Câu hỏi: {r['question']}\n")
    print("=== 🧠 Thinking ===")
    pprint(thinking_text)

    print("\n=== 💬 Final Answer ===")
    print(final_answer)

    print("\n=== 📚 Rationale ===")
    pprint(rationale_text)
    print("-" * 80)



❓ Câu hỏi: Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?

=== 🧠 Thinking ===
''

=== 💬 Final Answer ===


=== 📚 Rationale ===
''
--------------------------------------------------------------------------------

❓ Câu hỏi: Năm 2010, công nghệ nào thường được sử dụng trong kiểm soát ra vào ngôi nhà thông minh?

=== 🧠 Thinking ===
''

=== 💬 Final Answer ===


=== 📚 Rationale ===
''
--------------------------------------------------------------------------------

❓ Câu hỏi: Bộ cảm biến trong nhà thông minh thực hiện chức năng gì và bộ truyền động có vai trò gì?

=== 🧠 Thinking ===
''

=== 💬 Final Answer ===


=== 📚 Rationale ===
''
--------------------------------------------------------------------------------

❓ Câu hỏi: Nội dung chính của bài báo nghiên cứu Public_001 về IoT trong xây dựng nhà thông minh là gì?

=== 🧠 Thinking ===
''

=== 💬 Final Answer ===


=== 📚 Rationale ===
''
--------------------------------------------------------------------------------

❓ Câu hỏi:

In [11]:
for r in results:
    response = r["response"]

    # --- Tách phần <think> ... </think>
    thinking = re.search(r"<think>(.*?)</think>", response, re.DOTALL)
    thinking_text = thinking.group(1).strip() if thinking else ""

    # --- Phần còn lại sau </think>
    remainder = re.sub(r"<think>.*?</think>", "", response, flags=re.DOTALL).strip()

    # --- Tách rationale và final answer (nếu có)
    rationale_match = re.search(r"Rationale:(.*?)(?:Final Answer:|$)", remainder, re.DOTALL)
    rationale_text = rationale_match.group(1).strip() if rationale_match else ""

    final_match = re.search(r"Final Answer:\s*(.*)", remainder)
    final_answer = final_match.group(1).strip() if final_match else ""

    # --- In ra kết quả
    print(f"❓ Câu hỏi: {r['question']}")
    print("=== 💬 Final Answer ===")
    print(final_answer)
    print("-" * 80)

❓ Câu hỏi: Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?
=== 💬 Final Answer ===

--------------------------------------------------------------------------------
❓ Câu hỏi: Năm 2010, công nghệ nào thường được sử dụng trong kiểm soát ra vào ngôi nhà thông minh?
=== 💬 Final Answer ===

--------------------------------------------------------------------------------
❓ Câu hỏi: Bộ cảm biến trong nhà thông minh thực hiện chức năng gì và bộ truyền động có vai trò gì?
=== 💬 Final Answer ===

--------------------------------------------------------------------------------
❓ Câu hỏi: Nội dung chính của bài báo nghiên cứu Public_001 về IoT trong xây dựng nhà thông minh là gì?
=== 💬 Final Answer ===

--------------------------------------------------------------------------------
❓ Câu hỏi: Dựa vào thông tin trong tài liệu Public_001, hãy tính số bài báo trung bình mỗi lớp trong nghiên cứu được đề cập.
=== 💬 Final Answer ===

----------------------------------------------------------